# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display
import ollama

In [2]:
# open ai constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL_GPT = 'gpt-4o-mini'

API key looks good so far


In [3]:
# ollama constants

OLLAMA_API = "http://localhost:11434/api/chat"
#OLLAMA_HEADERS = {"Content-Type": "application/json"}
MODEL_LLAMA = 'llama3.2'

In [4]:
# initialization
openai = OpenAI()


In [20]:
# here is the question; type over this to ask something new
system_prompt="You are a tech specialist capable to explain with clarity tech question and more. You should answer in markdown"
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""
user_prompt = f"Please explain me this question: {question}"

In [21]:
# messages
prompt_messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ]

In [7]:
# Get gpt-4o-mini to answer, with streaming
def askOpenAi():
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=prompt_messages,
        stream = True        
    )
    response=""
    display_handle = display(Markdown(""), display_id=True)
    
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''  
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
askOpenAi()

Sure! Let's break down the code you've provided:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

### Explanation of the Code

1. **Set Comprehension:**
   - The code `{book.get("author") for book in books if book.get("author")}` is a **set comprehension**. This is a concise way to create a set in Python.
   - It iterates over a collection named `books`.

2. **Accessing the "author" Key:**
   - `book.get("author")`: For each `book` in the `books` collection, it attempts to get the value associated with the key `"author"`. 
   - The `get()` method is used to safely access dictionary keys. If the key does not exist, it returns `None` instead of throwing an error.

3. **Filtering Authors:**
   - The clause `if book.get("author")` acts as a filter. It ensures that only books with a valid (non-`None`) author get included in the resulting set.
   - Therefore, this part: `{book.get("author") for book in books if book.get("author")}` creates a set of unique authors from the `books` collection that have valid author values.

4. **Yielding Results:**
   - The `yield from` statement is used to yield values from the set comprehension created previously. 
   - This means that the containing function will return each unique author one at a time as they are requested (similar to a generator).

### Summary

- **What the Code Does:**
  - It generates a set of unique authors from a list of books, filtering out any entries that do not have an author. It then yields each of these authors.

- **Why It's Useful:**
  - This code is particularly useful when dealing with collections of books where some might not have an author specified. It safely retrieves the authors and ensures that each author is only returned once. 
  - Using `yield from` makes it memory efficient, as it does not create an intermediate list of authors but generates them one at a time on demand.

### Example

If you had a list of books like this:

```python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": None},
    {"title": "Book 4", "author": "Author A"},
]
```

The output of the code would be:

```
Author A
Author B
```

In this example, `Author A` is listed only once, even though there are multiple books by that author.

In [18]:
# Get Llama 3.2 to answer
def askOllama():    
    stream = ollama.chat(model=MODEL_LLAMA, messages=prompt_messages, stream=True)
    response=""
    display_handle = display(Markdown(""), display_id=True)
    
    for chunk in stream:
        response += chunk['message']['content'] or ''  
        update_display(Markdown(response), display_id=display_handle.display_id)
   

In [23]:
askOllama()

**Yielding Authors from a List of Books**
=====================================

This code snippet is written in Python and utilizes the `yield from` statement, which was introduced in Python 3.3.

### What does it do?

The code takes two main inputs:

*   A list of dictionaries (`books`) where each dictionary represents a book.
*   Another dictionary (`book`) that contains information about an author.

It generates a sequence of authors from the `books` list and yields them one by one, while also applying the condition that the book has a valid "author" key in its dictionary.

Here's a step-by-step breakdown:

1.  `{book.get("author") for book in books if book.get("author")}`:
    *   This is an expression that generates a sequence of authors.
    *   `for book in books` iterates over each book in the `books` list.
    *   `if book.get("author")` filters out books without an "author" key, to prevent errors and ensure only valid data is processed.

2.  `yield from ...`:
    *   This statement is used to delegate a sub-generator or iterator.
    *   In this case, it's delegating the sequence of authors generated in step 1.

**Why does it yield authors?**

The use of `yield from` serves two main purposes:

*   **Efficiency**: Instead of creating a new list with all the authors, this code yields each author one by one. This approach is more memory-efficient and can be particularly beneficial when dealing with large datasets.
*   **Flexibility**: By using `yield from`, you can create generators that produce values on-the-fly, allowing for lazy evaluation.

### Example Usage

Here's an example of how you might use this code:

```python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3"}
]

def get_authors(books):
    """Yields authors from the books."""
    yield from {book.get("author") for book in books if book.get("author")}

# Get all unique authors
authors = set(get_authors(books))
print(authors)  # Output: {'Author A', 'Author B'}
```

In this example, `get_authors` is a generator function that yields unique authors from the `books` list. The generated values are collected in a set (`authors`) to eliminate duplicates.